# Meteorological Data Visualization Tool / Инструмент визуализации метеорологических данных


In [1]:
#HIDDEN
%matplotlib inline

from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
from io import BytesIO
import datetime
import ipywidgets as widgets
from ipywidgets import widgets as wd
from IPython.display import display, clear_output, Markdown
from traitlets import traitlets

from scipy import constants
import math

import matplotlib.pyplot as plt
import pywt
from statsmodels.robust import mad
import seaborn as sns

sns.set(style='darkgrid')
output = widgets.Output()
interval_dict = {'1 second / 1 секунда': 'S', '1 minute / 1 минута': 'T', '10 minutes / 10 минут': '10T',
                 '30 minutes / 30 минут': '30T', '1 hour / 1 час': 'H', '12 hours / 12 часов': '12H',
                 '1 day / 1 день': 'D', '1 week / 1 неделя': 'W', '1 month / 1 месяц': 'M', 
                 '3 months / 3 месяца': 'Q', '6 months / 6 месяцев': '2Q', '1 year / 1 год': 'A'}

#Progress bar widget, from https://github.com/kuk/log-progress
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

#Special button class with variable 
#from https://stackoverflow.com/questions/45188464/return-output-of-the-function-executed-on-click
class LoadedButton(wd.Button):
    """A button that can holds a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))

#URL for the main directory:
url = 'http://amk030.imces.ru/meteodata/AMK_030_BIN/'

#Create a data type to extract the timestamp:
timestamp_type = np.dtype([('year', 'i2'),
                           ('month', 'i2'),
                           ('day', 'i2'),
                           ('hour', 'i2'),
                           ('minute', 'i2'),
                           ('second', 'i2'),
                           ('millisecond', 'i2')])

#Create a data type with the binary data format and the desired column names:
raw_data_type = np.dtype([('Temperature, °C / Температура, °C', 'i2'),
                          ('Southern Wind Speed, m/s / Скорость южного ветра, м/с', 'i2'),
                          ('Eastern Wind Speed, m/s / Скорость восточного ветра, м/с', 'i2'),
                          ('Vertical Wind Speed, m/s / Вертикальная скорость ветра, м/с', 'i2'),
                          ('Atmospheric Pressure, mmHg / Атмосферное давление, мм рт. ст', 'i2'),
                          ('Relative Air Humidity, % / Относительная влажность воздуха, %', 'i2'),
                          ('Error (0-9)', 'i1')])

#Function for creation the relevant links available in the main directory:
def MainDirectory(url):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    return [node.get('href').rstrip('/') 
            for node in soup.find_all('a') 
            if re.search(r'[0-9]/$|\)/$', node.get('href'))]

#Function for retrieving links to binary files,
#checking for the 'B' at the end:
def FileDirectory(url):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    return [url + '/' + node.get('href') 
            for node in soup.find_all('a') 
            if node.get('href').endswith('B')]

#Function for timestamps:
def Timestamps_Function(file, length):
    try:
        #raw_timestamp = np.frombuffer(file, dtype=timestamp_type, count=1)
        raw_timestamp = np.frombuffer(file[:14], dtype=timestamp_type)
        timestamp_elements = list(raw_timestamp[0].tolist())
        timestamp_elements[6] = timestamp_elements[6]*1000
        initial_date = datetime.datetime(*map(int, timestamp_elements))
        timestamps.append(initial_date)
        for i in range(length-1): 
            timestamps.append(timestamps[-1] + datetime.timedelta(milliseconds=12.5))
    except:
        pass

#Function for retrieving full data of each file:
def Data_Array(url):
    try:
        r = requests.get(url)
        file = r.content
        raw_data = file[17:-14]
        subarray = np.frombuffer(raw_data, dtype=raw_data_type)
        Timestamps_Function(file, len(subarray))
        return subarray
    except:
        pass

#Function for Wavelet Filtration, 
#from https://www.kaggle.com/jeffreyegan/vsb-power-line-fault-detection-approach/data

def waveletSmooth(x, wavelet="db4", level=10, title=None):
    # calculate the wavelet coefficients
    coeff = pywt.wavedec(x, wavelet, mode="per" )
    # calculate a threshold
    sigma = mad(coeff[-level])
    # changing this threshold also changes the behavior,
    # but I have not played with this very much
    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode="soft" ) for i in coeff[1:] )
    # reconstruct the signal using the thresholded coefficients
    y = pywt.waverec( coeff, wavelet, mode="per" )
    f, ax = plt.subplots()
    plt.plot( x, color="b", alpha=0.5 )
    plt.plot( y, color="r" )
    if title:
        ax.set_title(title)
    ax.set_xlim((0,len(y)))

#Create the lists:
available_years = MainDirectory(url)
selected_years = []
available_months = []
selected_months = []
file_directory = []
days_list = []
date_selection = []
timestamps = []
df = pd.DataFrame()


start_button = widgets.Button(description='Check available years / Проверить доступные годы',
                             layout=widgets.Layout(width='40%', height='60px'))
years_button = widgets.Button(description='Confirm years / Подтвердить годы',
                             layout=widgets.Layout(width='40%', height='60px'))
months_button = LoadedButton(description='Confirm months / Подтвердить месяцы',
                             layout=widgets.Layout(width='40%', height='60px'))
range_button = widgets.Button(description='Confirm range / Подтвердить диапазон времени',
                             layout=widgets.Layout(width='40%', height='60px'))
interval = widgets.RadioButtons(
    options=['1 second / 1 секунда', '1 minute / 1 минута', 
             '10 minutes / 10 минут', '30 minutes / 30 минут', 
             '1 hour / 1 час', '12 hours / 12 часов', '1 day / 1 день', 
             '1 week / 1 неделя', '1 month / 1 месяц', '3 months / 3 месяца', 
             '6 months / 6 месяцев', '1 year / 1 год'],
    value='10 minutes / 10 минут',
    description='Choose the data interval (how to group the data) / Выберите интервал данных (как сгруппировать данные):',
    layout={'width': 'initial'},
    style = {'description_width': 'initial'},
    disabled=False)
interval_button = LoadedButton(description='Confirm interval / Подтвердить интервал',
                             layout=widgets.Layout(width='40%', height='60px'))
fields = widgets.RadioButtons(
    options=['Temperature, °C / Температура, °C', 
             'Southern Wind Speed, m/s / Скорость южного ветра, м/с', 
             'Eastern Wind Speed, m/s / Скорость восточного ветра, м/с',
             'Vertical Wind Speed, m/s / Вертикальная скорость ветра, м/с', 
             'Atmospheric Pressure, mmHg / Атмосферное давление, мм рт. ст', 
             'Relative Air Humidity, % / Относительная влажность воздуха, %'],
    value='Temperature, °C / Температура, °C',
    description='Choose the field you want / Выберите поле, которое вы хотите:',
    layout={'width': 'initial'},
    style = {'description_width': 'initial'},
    disabled=False)
graph1_button = widgets.Button(description='Show graphs / Показать графики', 
                               layout=widgets.Layout(width='40%', height='60px'))
csv_button = widgets.Button(description='Export to CSV / Показать графики', 
                               layout=widgets.Layout(width='40%', height='60px'))
#graph2_button = widgets.Button(description='Show grouped data graph / Показать график сгруппированных данных', 
#                               layout=widgets.Layout(width='60%', height='60px'))


def start_button_clicked(_):
    for i in log_progress(range(len(available_years)), every=1):
        selected_years.append(widgets.Checkbox(description = available_years[i]))
        display(selected_years[i])
    display(years_button)

def years_button_clicked(_):
    for i in log_progress(range(len(available_years))):
        if selected_years[i].value == True:
            available_months.extend(MainDirectory(url + available_years[i] + '/'))
    for i in log_progress(range(len(available_months))):
        selected_months.append(widgets.Checkbox(description = available_months[i]))
        display(selected_months[i])
    display(months_button)
        
def months_button_clicked(button):
    for i in log_progress(range(len(available_months))):
        if selected_months[i].value == True:
            file_directory.extend(
                FileDirectory(
                    url + re.search(r'\d{4}', available_months[i]).group(0) 
                    + '/' + available_months[i] + '/'))
    days_aux = [datetime.date(int('20' + file_directory[i][-3:-1]),
                              int(file_directory[i][-12:-10]),
                              int(file_directory[i][-10:-8]))
                for i in log_progress(range(len(file_directory)))]
    date_selection = widgets.SelectionRangeSlider(
        options=days_aux,
        index=(0, len(days_aux)-1),
        #description='Select range / Выбрать диапазон времени:',
        disabled=False,
        layout={'width': 'initial'})
    days_list.extend(days_aux)    
    display(Markdown('Select range / Выбрать диапазон времени:'))
    display(date_selection)
    display(range_button)
    button.value = date_selection
    
def range_button_clicked(_):
    display(interval)
    display(interval_button)

    
def interval_button_clicked(button):
    date_selection = months_button.value.value
    frames = [Data_Array(i) for i in log_progress(file_directory[days_list.index(date_selection[0]):
                len(days_list) - 1 - days_list[::-1].index(date_selection[1])])]
    data = np.concatenate(frames)
    df = pd.DataFrame(data, columns=data.dtype.names)
    df['Temperature, °C / Температура, °C'] = df[
        'Temperature, °C / Температура, °C'].apply(lambda x: x/100)
    df['Southern Wind Speed, m/s / Скорость южного ветра, м/с'] = df[
        'Southern Wind Speed, m/s / Скорость южного ветра, м/с'].apply(lambda x: x/100)
    df['Eastern Wind Speed, m/s / Скорость восточного ветра, м/с'] = df[
        'Eastern Wind Speed, m/s / Скорость восточного ветра, м/с'].apply(lambda x: x/100)
    df['Vertical Wind Speed, m/s / Вертикальная скорость ветра, м/с'] = df[
        'Vertical Wind Speed, m/s / Вертикальная скорость ветра, м/с'].apply(lambda x: x/100)
    df['Atmospheric Pressure, mmHg / Атмосферное давление, мм рт. ст'] = df[
        'Atmospheric Pressure, mmHg / Атмосферное давление, мм рт. ст'].apply(lambda x: x/10)
    df['Relative Air Humidity, % / Относительная влажность воздуха, %'] = df[
        'Relative Air Humidity, % / Относительная влажность воздуха, %'].apply(lambda x: x/100)
    df['Timestamp / Временная метка'] = timestamps
    print(df.describe())
    display(Markdown('Correlation matrix / Корреляционная матрица:'))
    print(df.corr())
    plt.matshow(df.corr())
    plt.show()
    # <T>
    temperature_average = df['Temperature, °C / Температура, °C'].mean()
    # <P>
    pressure_average = df['Atmospheric Pressure, mmHg / Атмосферное давление, мм рт. ст'].mean()
    # <w>
    vertical_average = df['Vertical Wind Speed, m/s / Вертикальная скорость ветра, м/с'].mean()
    # <vs>
    southern_average = df['Southern Wind Speed, m/s / Скорость южного ветра, м/с'].mean()
    # <ve>
    eastern_average = df['Eastern Wind Speed, m/s / Скорость восточного ветра, м/с'].mean()
    # <V>
    average_wind = np.sqrt(southern_average ** 2 + eastern_average ** 2 + vertical_average ** 2)
    # <Vh>
    average_horizontal = np.sqrt(southern_average ** 2 + eastern_average ** 2)
    # ψ
    angle_inclination = np.arccos(average_horizontal / average_wind)
    # <D>
    f = np.arctan(-1 * eastern_average / southern_average)
    d = np.select([
        (southern_average > .01), 
        (southern_average < -.01) and (eastern_average >= 0), 
        (southern_average < -.01) and (eastern_average < 0), 
        (abs(southern_average) <= .01) and (eastern_average >= 0), 
        (abs(southern_average) <= .01) and (eastern_average < 0)], 
        [(constants.pi + f), (f), (2 * constants.pi + f), (constants.pi / 2), (3 * constants.pi / 2)])
    # ew
    df['Saturated vapor pressure / Давление насыщенного пара'] = (
        (6.112 * np.exp((17.62 * df['Temperature, °C / Температура, °C']) / (243.12 * df['Temperature, °C / Температура, °C']))) * (
        1.0016 + 3.15 * 10**(-6) * df['Atmospheric Pressure, mmHg / Атмосферное давление, мм рт. ст'] - 0.074 * df['Atmospheric Pressure, mmHg / Атмосферное давление, мм рт. ст']**(-1)))
    # e
    df['Elasticity (pressure) of water vapor / Упругость (давление) водяного пара'] = (
        df['Saturated vapor pressure / Давление насыщенного пара'] * df[
            'Relative Air Humidity, % / Относительная влажность воздуха, %']) / 100
    # Ed
    df['Moisture deficit / Дефицит влажности'] = df['Saturated vapor pressure / Давление насыщенного пара'] - df['Elasticity (pressure) of water vapor / Упругость (давление) водяного пара']
    # q
    df['Absolute air humidity / Абсолютная влажность воздуха'] = (df[
        'Relative Air Humidity, % / Относительная влажность воздуха, %'] * df[
        'Atmospheric Pressure, mmHg / Атмосферное давление, мм рт. ст'] * 28.9647) / (constants.R * df[
        'Temperature, °C / Температура, °C'] * 100)
    # p
    df['Air density / Плотность воздуха'] = (df[
        'Atmospheric Pressure, mmHg / Атмосферное давление, мм рт. ст'] * 28.9647) / (constants.R * df[
        'Temperature, °C / Температура, °C'])
    # T'
    df['Temperature pulsations / Температурные пульсации'] = df[
        'Temperature, °C / Температура, °C'] - temperature_average
    # w'
    df['Vertical components / Вертикальные компоненты'] = df[
        'Vertical Wind Speed, m/s / Вертикальная скорость ветра, м/с'] - vertical_average
    # u'
    df['Longitudinal components / Продольные компоненты'] = (southern_average * (df[
        'Southern Wind Speed, m/s / Скорость южного ветра, м/с'] - southern_average) + eastern_average * (df[
        'Eastern Wind Speed, m/s / Скорость восточного ветра, м/с'] - eastern_average) / average_horizontal)
    # v'
    df['Transversal components / Поперечные компоненты'] = ((southern_average * -1) * (df[
        'Eastern Wind Speed, m/s / Скорость восточного ветра, м/с'] - eastern_average) + eastern_average * (df[
        'Southern Wind Speed, m/s / Скорость южного ветра, м/с'] - southern_average) / average_horizontal)
    # σ_u^2, σ_v^2 , σ_w^2, σ_t^2
    longitudinal_var = df['Longitudinal components / Продольные компоненты'].var()
    transversal_var = df['Transversal components / Поперечные компоненты'].var()
    vertical_var = df['Vertical components / Вертикальные компоненты'].var()
    temperature_var = df['Temperature pulsations / Температурные пульсации'].var()
    # 〈T' * w'〉
    momentum_heat = (df['Temperature pulsations / Температурные пульсации'] * df[
        'Vertical components / Вертикальные компоненты']).mean()
    # 〈u' * w'〉
    momentum_pulse = (df['Longitudinal components / Продольные компоненты'] * df[
        'Vertical components / Вертикальные компоненты']).mean()
    # Ev
    ev = (longitudinal_var + transversal_var + vertical_var) / 2
    # lv
    lv = ev / (average_wind ** 2)
    # Et
    et = temperature_var / 2
    # τ
    df['Vertical impulse flow / Вертикальный поток импульса'] = -1 * df[
        'Air density / Плотность воздуха'] * momentum_pulse
    # v*
    friction_velocity = math.sqrt(abs(momentum_pulse))
    # T*
    temperature_scale = -1 * momentum_heat
    # L*
    mo_scale = temperature_average * (friction_velocity ** 2) / 0.4 * 9.81 * temperature_scale
    # other variables
    average_turbotemp = df['Temperature pulsations / Температурные пульсации'].mean()
    average_longcomp = df['Longitudinal components / Продольные компоненты'].mean()
    # Constants:
    cd = (friction_velocity / average_wind) ** 2
    ct2 = ((average_turbotemp - df['Temperature pulsations / Температурные пульсации'] - temperature_average) *
          (average_wind ** (-2/3))).mean()
    cv2 = ((average_longcomp - ((southern_average * (df[
        'Southern Wind Speed, m/s / Скорость южного ветра, м/с'] - southern_average) + eastern_average * (df[
        'Eastern Wind Speed, m/s / Скорость восточного ветра, м/с'] - eastern_average)) / d)) * (average_wind ** (-2/3))).mean()
    cna2 = (ct2 / (4 *((temperature_average + 273.15) ** 2))) + (cv2 / (343 ** 2))
    cno2 = ((8 * (10**-5) * pressure_average / (temperature_average**2))**2) * ct2
    height = 27
    dimensionless_height = height / mo_scale
    #φv
    fv = (1 + 4.7 * dimensionless_height) if dimensionless_height > 0 else (1 - 15 * dimensionless_height) ** .25
    #φt
    ft = (0.74 + 4.7 * dimensionless_height) if dimensionless_height > 0 else (1 - 15 * dimensionless_height) ** .25
    dudz = (friction_velocity / 0.4) * fv * (height / mo_scale)
    dodz = (temperature_scale / 0.4) * ft * (height / mo_scale)
    dtdz = dodz - .0098
    km = ((friction_velocity)**2) / dudz
    kh = (friction_velocity * temperature_scale) / dodz
    lo = (km / abs(dudz))**0.5
    energy_dissipation_wind = (km * (dudz**2)) - (9.81 / temperature_average) * kh * dodz
    energy_dissipation_temperature = kh * (dudz**2)
    richardson = (9.81 / temperature_average) * dodz / dudz
    df = df.dropna(how='any')
    output = """                       ____________________
                        характеристики воздуха:
                    
    {} <= T - средняя температура, градусы Цельсия;
    {} <= RMS(T) - стандартное отклонение температуры, градусы Цельсия;
    {} <= P - атмосферное давление, мм.рт.ст.;
    {} <= r - относительная влажность воздуха, %;
    {} <= e - упругость (давление) водяного пара, гПа;
    {} <= Ed - дефицит влажности, гПа;
    {} <= ro - плотность воздуха, кг/м^3;
                       ____________________
                       характеристики ветра:
                   
    {} <= V - средняя скорость горизонтального ветра, м/с;
    {} <= D - среднее направление горизонтального ветра, градусы;
    {} <= w - средняя скорость вертикального ветра, м/с;
    {} <= RMS(w) - стандартное отклонение скорости вертикального ветра, м/с;
    {} <= Vm - модуль среднего вектора скорости ветра, м/с;
    {} <= Vs - среднее значение южной компоненты горизонтального ветра, м/с;
    {} <= Ve - среднее значение восточной компоненты горизонтального ветра, м/с.

                       ________________________
                       ПАРАМЕТРЫ ТУРБУЛЕНТНОСТИ:

    {} <= Ev - полная энергия турбулентных движений, (м^2)/(с^2);
    {} <= Iv - относительная интенсивность флуктуаций скорости ветра;
    {} <= Et - энергия температурных флуктуаций, град^2
    {} <= <u*w> - момент потока импульса, (м^2)/(с^2);
    {} <= <w*T> - момент потока тепла, (м^2)/(с^2);
    {} <= Tau - вертикальный поток импульса;
    {} <= Cd - коэффициент сопротивления потоку;
    {} <= T* - масштаб температуры, град;
    {} <= L* - масштаб Монина-Обухова, м;
    {} <= Ct2 - структ. постоянная флукт. температуры, град^2/(см^2/3);
    {} <= Cv2 - структ. постоянная флуктуаций ветра, (м/с)^2/(см^2/3);
    {} <= Cn2a - структ. пост.для акустич. показат. преломл., 1/(м^2/3);
    {} <= Cn2o - структ. пост. для оптич. показат. преломл., 1/(см^2/3);
    {} <= Локальный градиент скорости ветра;
    {} <= Локальный градиент потенциальной температуры;
    {} <= Локальный градиент температуры;
    {} <= Коэффициент турбулентного обмена количества движения;
    {} <= Коэффициент турбулентного обмена тепла;
    {} <= Внешний масштаб турбулентности;
    {} <= Скорость диссипации энергии ветровых флуктуаций;
    {} <= Скорость диссипации энергии температурных;
    {} <= Градиентное число Ричардсона
    """.format(temperature_average, 
               math.sqrt(temperature_var), 
               pressure_average, 
               df['Relative Air Humidity, % / Относительная влажность воздуха, %'].mean(), 
               df['Elasticity (pressure) of water vapor / Упругость (давление) водяного пара'].mean(), 
               df['Moisture deficit / Дефицит влажности'].mean(),
               df['Absolute air humidity / Абсолютная влажность воздуха'].mean(),
               df['Air density / Плотность воздуха'].mean(),
               average_horizontal,
               d,
               vertical_average,
               df['Vertical Wind Speed, m/s / Вертикальная скорость ветра, м/с'].var(),
               average_wind,
               southern_average,
               eastern_average,
               ev,
               lv,
               et,
               momentum_pulse,
               momentum_heat,
               df['Vertical impulse flow / Вертикальный поток импульса'].mean(),
               cd,
               temperature_scale,
               mo_scale,
               ct2,
               cv2,
               cna2,
               cno2,
               dudz,
               dodz,
               dtdz,
               km,
               kh,
               lo,
               energy_dissipation_wind,
               energy_dissipation_temperature,
               richardson
              )
    print(output)
    display(fields)
    display(graph1_button)
    button.value = df

def graph1_button_clicked(_):
    df = interval_button.value
    copy = df.copy()
    waveletSmooth(df[fields.value], title=fields.value +
                           ',\nFull Data with Daubechies 4 Wavelet Filtration /' +
                           '\nПолные данные с вейвлет-фильтрацией Daubechies 4')
    plt.show()
    copy.set_index('Timestamp / Временная метка', inplace = True)
    copy[fields.value].resample(
        interval_dict[interval.value]).mean().dropna().plot(title='Resampled Data / Передискретизация данных')
    plt.show()
    print('First record / Первая запись:')
    print(copy.index[0])
    print('Last record / Последняя запись:')
    print(copy.index[-1])
    display(csv_button)
    #print(df.corr())
    #plt.matshow(df.corr())
    #plt.show()
    #display(graph2_button)
    
#def graph2_button_clicked(_):
#    interval_button.value[fields.value].resample(
#        interval_dict[interval.value]).mean().dropna().plot(title='Resampled Data / Передискретизация данных')

def csv_button_clicked(_):
    interval_button.value.to_csv('Extracted_data.csv')

start_button.on_click(start_button_clicked)
display(start_button)

years_button.on_click(years_button_clicked)
months_button.on_click(months_button_clicked)
interval_button.on_click(interval_button_clicked)
range_button.on_click(range_button_clicked)
graph1_button.on_click(graph1_button_clicked)
csv_button.on_click(csv_button_clicked)
#graph2_button.on_click(graph2_button_clicked)

Button(description='Check available years / Проверить доступные годы', layout=Layout(height='60px', width='40%…